In [ ]:
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
import os
from itertools import combinations
from sklearn.metrics import cohen_kappa_score, f1_score


warnings.simplefilter('ignore')
tqdm.pandas()

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
pd.set_option('max_colwidth', 200)

In [ ]:
seed = 2020

In [ ]:
df_feature = pd.read_pickle('data/user.pkl')

df_feature['label'] = df_feature['label'] + 1
df_feature.head()

In [ ]:
df_feature['label'].value_counts()

In [ ]:
# 1.0    92818
# 0.0    30237
# 2.0    22241

In [ ]:
all_users = list(set(df_feature['cust_no'].values))

In [ ]:
len(all_users)

# 特征工程

In [ ]:
df_feature['fq'] = df_feature['q']
df_feature.loc[df_feature['fq'] == 1, 'fq'] = 5

## 行为信息

In [ ]:
df_behavior = pd.read_pickle('data/behavior.pkl')
print(df_behavior.shape)
df_behavior = df_behavior[df_behavior['cust_no'].isin(all_users)]
print(df_behavior.shape)
df_behavior.sort_values(['cust_no', 'q', 'm'], inplace=True)
df_behavior.head()

In [ ]:
# 单独提取B6
df_temp = df_behavior[['cust_no', 'q', 'B6']]
df_temp = df_temp.dropna()
df_temp.head()

# 只保留三季度大于2019.7.1的数据
# 只保留四季度大于2019.10.1的数据
df_temp1 = df_temp[(df_temp['q'] == 3) & (
    df_temp['B6'] >= '2019-07-01 00:00:00')]
df_temp2 = df_temp[(df_temp['q'] == 4) & (
    df_temp['B6'] >= '2019-10-01 00:00:00')]
df_temp3 = df_temp[(df_temp['q'] == 1)]

df_temp = pd.concat([df_temp1, df_temp2, df_temp3])
df_temp = df_temp.reset_index(drop=True)

df_temp['last'] = 0
df_temp.loc[df_temp['q'] == 1, 'last'] = '2020-04-01 00:00:00'
df_temp.loc[df_temp['q'] == 3, 'last'] = '2019-10-01 00:00:00'
df_temp.loc[df_temp['q'] == 4, 'last'] = '2020-01-01 00:00:00'

df_temp['B6_gap'] = (pd.to_datetime(df_temp['last']) -
                     pd.to_datetime(df_temp['B6'])).dt.total_seconds()

del df_temp['B6'], df_temp['last']
df_feature = df_feature.merge(df_temp, how='left')

In [ ]:
# 单独提取B7
df_temp = df_behavior[['cust_no', 'q', 'B7']]
df_temp = df_temp.dropna()
df_feature = df_feature.merge(df_temp, how='left')

In [ ]:
df_behavior['B5_B3_minus'] = df_behavior['B5'] - df_behavior['B3']
df_behavior['B5_B3_ratio'] = df_behavior['B5'] / (df_behavior['B3'] + 1e-3)
df_behavior['B3_B2_ratio'] = df_behavior['B3'] / df_behavior['B2']
df_behavior['B5_B4_ratio'] = df_behavior['B5'] / df_behavior['B4']

In [ ]:
for f in tqdm(['B1', 'B2', 'B3', 'B4', 'B5', 'B5_B3_minus', 'B3_B2_ratio', 'B5_B4_ratio', 'B5_B3_ratio']):
    df_temp = df_behavior.groupby(['cust_no', 'q'])[f].agg({
        'q_{}_mean'.format(f): 'mean',
        'q_{}_std'.format(f): 'std',
        'q_{}_max'.format(f): 'max',
        'q_{}_min'.format(f): 'min',
        'q_{}_diff'.format(f): lambda x: x.values[-1] - x.values[0],
        'q_{}_last'.format(f): 'last',
    }).reset_index()
    df_feature = df_feature.merge(df_temp, how='left')
df_feature.head()

## 资产信息

In [ ]:
df_aum = pd.read_pickle('data/aum.pkl')
print(df_aum.shape)
df_aum = df_aum[df_aum['cust_no'].isin(all_users)]
print(df_aum.shape)
df_aum.sort_values(['cust_no', 'q', 'm'], inplace=True)
df_aum.head()

In [ ]:
X_cols = [f for f in df_aum.columns if f.startswith('X')]

df_aum['X_sum'] = df_aum[X_cols].sum(axis=1)
df_aum['X_num'] = (df_aum[X_cols] > 0).sum(axis=1)

In [ ]:
for f in tqdm(X_cols + ['X_sum', 'X_num']):
    df_temp = df_aum.groupby(['cust_no', 'q'])[f].agg({
        'q_{}_mean'.format(f): 'mean',
        'q_{}_std'.format(f): 'std',
        'q_{}_max'.format(f): 'max',
        'q_{}_min'.format(f): 'min',
        'q_{}_median'.format(f): 'median',
        'q_{}_diff'.format(f): lambda x: x.values[-1] - x.values[0],
        'q_{}_last'.format(f): 'last',
    }).reset_index()
    df_feature = df_feature.merge(df_temp, how='left')
df_feature.head()

## 存款信息

In [ ]:
df_cunkuan = pd.read_pickle('data/cunkuan.pkl')
print(df_cunkuan.shape)
df_cunkuan = df_cunkuan[df_cunkuan['cust_no'].isin(all_users)]
print(df_cunkuan.shape)
df_cunkuan.sort_values(['cust_no', 'q', 'm'], inplace=True)
df_cunkuan.head()

In [ ]:
df_cunkuan['C3'] = df_cunkuan['C1'] / df_cunkuan['C2']
df_cunkuan['C1_m_diff'] = df_cunkuan.groupby(['cust_no', 'q'])['C1'].diff()

In [ ]:
for f in ['C1', 'C2', 'C3', 'C1_m_diff']:
    df_temp = df_cunkuan.groupby(['cust_no', 'q'])[f].agg({
        'q_{}_mean'.format(f): 'mean',
        'q_{}_std'.format(f): 'std',
        'q_{}_max'.format(f): 'max',
        'q_{}_min'.format(f): 'min',
        'q_{}_median'.format(f): 'median',
        'q_{}_diff'.format(f): lambda x: x.values[-1] - x.values[0],
        'q_{}_last'.format(f): 'last',
    }).reset_index()
    df_feature = df_feature.merge(df_temp, how='left')
df_feature.head()

## 历史事件数据

In [ ]:
df_event = pd.read_pickle('data/event.pkl')
print(df_event.shape)
df_event = df_event[df_event['cust_no'].isin(all_users)]
print(df_event.shape)
df_event.head()

In [ ]:
datetime_feats = ['E{}'.format(i) for i in range(1, 15)] + ['E16', 'E18']

for f1, f2 in tqdm(list(combinations(datetime_feats, 2))):
    df_event['{}_{}_days_diff'.format(f1, f2)] = (pd.to_datetime(
        df_event[f1]) - pd.to_datetime(df_event[f2])).dt.days

In [ ]:
df_event['E17_E15_minus'] = df_event['E17'] - df_event['E15']

In [ ]:
df_feature = df_feature.merge(df_event, how='left')
df_feature.head()

## 基本信息

In [ ]:
df_cust_info = pd.read_pickle('data/cust_info.pkl')
print(df_cust_info.shape)
df_cust_info = df_cust_info[df_cust_info['cust_no'].isin(all_users)]
print(df_cust_info.shape)
df_cust_info.head()

In [ ]:
df_feature = df_feature.merge(df_cust_info, how='left')
df_feature.head()

In [ ]:
cate_features = ['I1', 'I3', 'I4', 'I5',
                 'I6', 'I8', 'I10', 'I13', 'I14']

for f in tqdm(cate_features):
    df_feature['{}_cnt'.format(f)] = df_feature.groupby([f])[
        'cust_no'].transform('count')

# 其他特征

In [ ]:
df_feature.sort_values(['cust_no', 'fq'], inplace=True)

In [ ]:
for f in ['q_{}_last'.format(f) for f in ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8']]:
    df_feature['{}_ratio'.format(f)] = df_feature[f] / \
        df_feature['q_X_sum_last']

In [ ]:
cols = ['q_{}_last'.format(f) for f in ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X_sum',
                                        'X_num']] + \
    ['q_{}_last'.format(f) for f in ['B1', 'B2', 'B3', 'B4', 'B5']]

for f in tqdm(cols):
    df_feature['{}_q_between_diff'.format(f)] = df_feature.groupby([
        'cust_no'])[f].diff()

In [ ]:
df_feature.to_pickle('data/feature.pkl')